In [55]:

import pandas as pd
import numpy as np

In [56]:
df = pd.read_csv('../Data/dontpatronizeme_pcl.tsv', sep = '\t', names=['id','info','country', 'text','label'] )

In [57]:
df = df.dropna(inplace = False)

df = df.reset_index(drop = True)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10468 entries, 0 to 10467
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       10468 non-null  object
 1   info     10468 non-null  object
 2   country  10468 non-null  object
 3   text     10468 non-null  object
 4   label    10468 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 409.0+ KB


In [58]:
df_final = df[['text','label']]
df_final
df_final['label_']= [ 0 if (y == 1 or y == 0) else 1 for y in df_final['label']]
df_final
df_final.drop('label', axis = 1, inplace= True)
df_final.head()

C:\Users\PREETA~1\AppData\Local\Temp/ipykernel_13492/1084671562.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['label_']= [ 0 if (y == 1 or y == 0) else 1 for y in df_final['label']]
C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:4906: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().drop(


,text,label_
0,"We 're living in times of absolute insanity , ...",0
1,"In Libya today , there are countless number of...",0
2,White House press secretary Sean Spicer said t...,0
3,Council customers only signs would be displaye...,0
4,""" Just like we received migrants fleeing El Sa...",0


In [59]:
X = df_final.drop('label_', axis =1) #getting independent variable

In [60]:
y = df_final['label_']

In [61]:
y.value_counts()

0    9475
1     993
Name: label_, dtype: int64

In [62]:
X.shape


(10468, 1)

In [63]:
import tensorflow as tf

In [64]:
tf.__version__

'2.7.0'

In [65]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional

In [71]:
voc_size = 10000

In [72]:
messages = X.copy()

In [73]:
messages['text'][0]

"We 're living in times of absolute insanity , as I 'm pretty sure most people are aware . For a while , waking up every day to check the news seemed to carry with it the same feeling of panic and dread that action heroes probably face when they 're trying to decide whether to cut the blue or green wire on a ticking bomb -- except the bomb 's instructions long ago burned in a fire and imminent catastrophe seems the likeliest outcome . It 's hard to stay that on-edge for that long , though , so it 's natural for people to become inured to this constant chaos , to slump into a malaise of hopelessness and pessimism ."

In [74]:
messages.reset_index(inplace= True)


In [75]:
import nltk
import re
from nltk.corpus import stopwords

In [76]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Preetam
[nltk_data]     Singh\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [77]:
from nltk.stem.porter import PorterStemmer

ps = PorterStemmer()
corpus =[]
from tqdm import tqdm 
for i in tqdm(range(0,10468)):
    
    review = re.sub('[^a-zA-Z]',' ', messages['text'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words ('english')]
    review =' '.join(review)
    corpus.append(review)
    

100%|███████████████████████████████████████████████████████████████████████████| 10468/10468 [01:04<00:00, 162.72it/s]


In [25]:
#corpus

['live time absolut insan pretti sure peopl awar wake everi day check news seem carri feel panic dread action hero probabl face tri decid whether cut blue green wire tick bomb except bomb instruct long ago burn fire immin catastroph seem likeliest outcom hard stay edg long though natur peopl becom inur constant chao slump malais hopeless pessim',
 'libya today countless number ghanaian nigerian immigr two countri key macroeconom challeng includ unemploy let tackl issu root fruit thank',
 'white hous press secretari sean spicer said focu immigr action would illeg immigr also otherwis violat law',
 'council custom sign would display two space would reserv disabl person would five p space eight p one',
 'like receiv migrant flee el salvador guatemala like seen thousand econom refuge start see mexican flee violenc ju rez part mexico say shelter director ruben garcia',
 'bring high blood sugar level insulin need taken type requir insulin meal time take correct dose insulin order lower blood

['we re living in times of absolute insanity as i m pretty sure most people are aware for a while waking up every day to check the news seemed to carry with it the same feeling of panic and dread that action heroes probably face when they re trying to decide whether to cut the blue or green wire on a ticking bomb except the bomb s instructions long ago burned in a fire and imminent catastrophe seems the likeliest outcome it s hard to stay that on edge for that long though so it s natural for people to become inured to this constant chaos to slump into a malaise of hopelessness and pessimism',
 'in libya today there are countless number of ghanaian and nigerian immigrants these are the two countries with key macroeconomic challenges including unemployment let s tackle this issue from the root and not the fruit thank you',
 'white house press secretary sean spicer said that the focus of immigration actions would be illegal immigrants who have also otherwise violated our laws',
 'council 

In [78]:
onehot_repr = [one_hot(words, voc_size)for words in corpus]
onehot_repr

[[8763,
  5041,
  437,
  4037,
  896,
  6419,
  8618,
  6414,
  2433,
  6359,
  8759,
  1380,
  7825,
  9459,
  2498,
  4037,
  9992,
  2528,
  3011,
  10,
  2056,
  3444,
  2220,
  3792,
  6069,
  3384,
  5928,
  7387,
  3853,
  2065,
  3664,
  3470,
  3664,
  3890,
  8917,
  2288,
  842,
  4848,
  9861,
  9035,
  9459,
  7184,
  8045,
  1716,
  1537,
  9813,
  8917,
  496,
  2702,
  8618,
  6878,
  7352,
  5007,
  4301,
  3378,
  635,
  4088,
  8753],
 [9225,
  8565,
  3376,
  3972,
  6099,
  1631,
  2527,
  7134,
  5265,
  4900,
  5331,
  3438,
  3010,
  9430,
  8259,
  4708,
  1186,
  3378,
  4288,
  7206],
 [2372,
  6493,
  1393,
  951,
  5670,
  5102,
  8784,
  2096,
  2527,
  3011,
  9809,
  1158,
  2527,
  5434,
  6932,
  5739,
  4809],
 [559,
  9681,
  6382,
  9809,
  228,
  7134,
  9616,
  9809,
  4235,
  1554,
  3549,
  9809,
  4597,
  3288,
  9616,
  5582,
  3288,
  6090],
 [9338,
  9709,
  7454,
  9199,
  7516,
  932,
  3293,
  9338,
  7358,
  4058,
  229,
  2545,
  6835,


In [79]:
sent_length = 60
embedded_docs= pad_sequences(onehot_repr, padding = 'pre', maxlen = sent_length)
print(embedded_docs)

[[   0    0 8763 ...  635 4088 8753]
 [   0    0    0 ... 3378 4288 7206]
 [   0    0    0 ... 6932 5739 4809]
 ...
 [   0    0    0 ... 8379 3325 2639]
 [   0    0    0 ... 5275  366 8300]
 [   0    0    0 ... 8708 2322 8759]]


In [80]:
embedded_docs[0]

array([   0,    0, 8763, 5041,  437, 4037,  896, 6419, 8618, 6414, 2433,
       6359, 8759, 1380, 7825, 9459, 2498, 4037, 9992, 2528, 3011,   10,
       2056, 3444, 2220, 3792, 6069, 3384, 5928, 7387, 3853, 2065, 3664,
       3470, 3664, 3890, 8917, 2288,  842, 4848, 9861, 9035, 9459, 7184,
       8045, 1716, 1537, 9813, 8917,  496, 2702, 8618, 6878, 7352, 5007,
       4301, 3378,  635, 4088, 8753])

In [81]:
embedding_vector_features = 50
model =Sequential()
model.add(Embedding(voc_size,embedding_vector_features, input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation ='sigmoid'))
model.compile(loss='binary_crossentropy', sample_weight_mode ='balanced',optimizer ='adam', metrics=['accuracy'])
print(model.summary())

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 60, 50)            500000    
                                                                 
 lstm_2 (LSTM)               (None, 100)               60400     
                                                                 
 dense_2 (Dense)             (None, 1)                 101       
                                                                 
Total params: 560,501
Trainable params: 560,501
Non-trainable params: 0
_________________________________________________________________
None


In [82]:
embedding_vector_features = 50
model1 =Sequential()
model1.add(Embedding(voc_size,embedding_vector_features, input_length = sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dense(1,activation ='sigmoid'))
model1.compile(loss='binary_crossentropy',  sample_weight_mode ='balanced',optimizer ='adam', metrics=['accuracy'])
print(model1.summary())

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (None, 60, 50)            500000    
                                                                 
 bidirectional_1 (Bidirectio  (None, 200)              120800    
 nal)                                                            
                                                                 
 dense_3 (Dense)             (None, 1)                 201       
                                                                 
Total params: 621,001
Trainable params: 621,001
Non-trainable params: 0
_________________________________________________________________
None


In [83]:
len(embedded_docs), y.shape

(10468, (10468,))

In [84]:
X_final= np.array(embedded_docs)
y_final=np.array(y)

In [85]:
X_final.shape, y_final.shape

((10468, 60), (10468,))

In [46]:
#weighted array
wt_array =len(X_final)/(len(set(y_final))*(np.bincount(y_final)))
wt_array

array([0.55240106, 5.27089627])

In [86]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size = 0.20, random_state= 0)

In [87]:
y_train.shape, y_test.shape

((8374,), (2094,))

In [88]:
X_train

array([[   0,    0,    0, ..., 3026,  124, 9590],
       [   0,    0,    0, ..., 3827, 4679, 2372],
       [   0,    0,    0, ..., 1188, 1978, 2663],
       ...,
       [   0,    0,    0, ..., 2037, 1733, 7789],
       [   0,    0,    0, ..., 4721, 5696, 3994],
       [   0,    0,    0, ..., 6746, 5350, 3771]])

In [50]:
model.fit(X_train, y_train, validation_data =(X_test, y_test), epochs=1, batch_size =64,)

131/131 [==============================] - 6s 47ms/step - loss: 0.0371 - accuracy: 0.9876 - val_loss: 0.5434 - val_accuracy: 0.8668


In [89]:
model1.fit(X_train, y_train, validation_data =(X_test, y_test), epochs=1, batch_size =64,)

131/131 [==============================] - 13s 82ms/step - loss: 0.3258 - accuracy: 0.8995 - val_loss: 0.2778 - val_accuracy: 0.9026


In [90]:

y_pred = model.predict(X_test)
classes_x=np.argmax(y_pred, axis =1)

In [91]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, classes_x)
print(cm)

[[1890    0]
 [ 204    0]]


In [92]:
accuracy_score(y_test, classes_x)

0.9025787965616046

In [93]:
from sklearn.metrics import classification_report
print(classification_report(y_test, classes_x))

              precision    recall  f1-score   support

           0       0.90      1.00      0.95      1890
           1       0.00      0.00      0.00       204

    accuracy                           0.90      2094
   macro avg       0.45      0.50      0.47      2094
weighted avg       0.81      0.90      0.86      2094



C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\ProgramData\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
